## Generate leap report for all facilities

ValueError: Length of values (28) does not match length of index (32)

Solution:
First, Run clear content leap macro on excel sheet then save the excel file where script is.
- If there is wrong dates on the data set, it is likely that only wrong data will be displayed. SOLUTION: Ensure all dates in the columns are the same or delete contents of the troublesome column.

In [6]:
%%time
import pandas as pd

# display all columns
pd.options.display.max_columns = None

# display all columns
pd.options.display.max_rows = None

path = 'LEAP TEMPLATE FY21 Q4 WK 7 UPPER KIAMBU.xlsx'


# Create a function
def create_leap_data(df, facility):
    #     print(facility)

    # read from the 3rd
    df = df.iloc[1:]
    # Convert 1st row to column header
    df.columns = df.iloc[0]
    # Reset index
    df.reset_index(drop=True, inplace=True)

    # read fron the 3rd
    df = df.iloc[1:]

    # fill all nan with zero
    df = df.fillna(0)

    # drop all columns with zeros
    df = df.loc[:, df.any()]

    # drop all columns with "WEEK"
    df = df[df.columns.drop(list(df.filter(regex='WEEK')))]
    df = df[df.columns.drop(list(df.filter(regex='week')))]

    # rename the first column name
    df.rename(columns={df.columns[0]: "indicator"}, inplace=True)

    # create same indicators
    mylist = ['Tested (HTS_TST)', 'Tested positive (HTS_POS)', 'Tested under index testing (HTS_INDEX)',
              'Tested positive under index testing (HTS_POS_INDEX)', 'Self testing kits issued (HTS_SELF)',
              '1st ANC Clients  seen (PMTCT_STAT)', '1st ANC Clients- KP', '1st ANC Clients new positive',
              '1st  ANC  POS + KP (PMTCT POS)', '1st ANC  on ART(PMTCT ART)', 'TB cases new (TB STAT)',
              'TB cases new positive',
              'TB cases  KP', 'TB cases  positive+KP (TB POS)', 'TB cases on ART (TB ART)', "0",
              'Initiated on ART (TX_NEW)',
              'New on ART initiated  on TPT', 'Returning to care (TX_RTC)', 'Transfer In (TI)',
              'Current on ART (TX_CURR)',
              'TXML-treatment mortality and loss(ltfu+TO+Dead)', 'LTFU(>30 days)', 'Transfer out(TO)', 'Deceased',
              'PREP_New', 'PREP_CURR',
              'GBV cases ', 'CXCA']

    mylist1 = ['Tested (HTS_TST)', 'Tested positive (HTS_POS)', 'Tested under index testing (HTS_INDEX)',
               'Tested positive under index testing (HTS_POS_INDEX)', 'Self testing kits issued (HTS_SELF)',
               '1st ANC Clients  seen (PMTCT_STAT)', '1st ANC Clients- KP', '1st ANC Clients new positive',
               '1st  ANC  POS + KP (PMTCT POS)', '1st ANC  on ART(PMTCT ART)', 'TB cases new (TB STAT)',
               'TB cases new positive',
               'TB cases  KP', 'TB cases  positive+KP (TB POS)', 'TB cases on ART (TB ART)',
               'Initiated on ART (TX_NEW)',
               'New on ART initiated  on TPT', 'Returning to care (TX_RTC)', 'Transfer In (TI)',
               'Current on ART (TX_CURR)',
               'TXML-treatment mortality and loss(ltfu+TO+Dead)', 'LTFU(>30 days)', 'Transfer out(TO)', 'Deceased',
               'PREP_New', 'PREP_CURR',
               'GBV cases ', 'CXCA']

    # create a new column
    try:
        df['new_col'] = mylist
    except:
        df['new_col'] = mylist1

    # replicate new column
    df['indicator'] = df['new_col']
    # drop new column
    df = df.drop('new_col', axis=1)
    # fill all columns with nan with "type"
    df.columns = df.columns.fillna('type')
    # drop all columns with "type"
    df = df[df.columns.drop(list(df.filter(regex='type')))]
    # df.head()

    # set index to indicator
    df.set_index('indicator', inplace=True)

    # transpose df
    df = df.T

    # Select column names to view
    col_needed = ['Tested (HTS_TST)', 'Tested positive (HTS_POS)', 'Tested under index testing (HTS_INDEX)',
                  'Tested positive under index testing (HTS_POS_INDEX)', 'Self testing kits issued (HTS_SELF)',
                  '1st ANC Clients  seen (PMTCT_STAT)', '1st ANC Clients  seen (PMTCT_STAT)','1st ANC Clients- KP', 
                  '1st ANC Clients new positive', '1st ANC  on ART(PMTCT ART)','TB cases new (TB STAT)', 
                  'TB cases new positive','TB cases  KP', 'TB cases on ART (TB ART)', 'PREP_New', 'PREP_CURR', 
                  'GBV cases ','Initiated on ART (TX_NEW)', 'Current on ART (TX_CURR)', 'CXCA',
                  'Returning to care (TX_RTC)', 'Transfer In (TI)', 'LTFU(>30 days)', 'Transfer out(TO)', 'Deceased']
    df = df[col_needed]
    
    # drop all rows with zeros
    df = df.loc[~(df == 0).all(axis=1)]
    
    # insert facility name as the first column
    df.insert(0, 'Facility', facility)
    return df


# create a list of facilities on leap
myFacilities = ['KCRH', 'LIMURU', 'WANGIGE', 'GITHUNGURI', 'KARURI', 'KIHARA', 'LARI', 'KIGUMO', 'GICHURU', 'LUSIGETTI',
                'NGEWA','UTHIRU', 'GITHIGA', 'Thika L5', 'Ruiru', 'Gatundu L5H', 'kiandutu',
                'langata','igegania', 'Ngoliba', 'jkuat', 'Makongeni', 'Kalimoni', 'Gachororo', 'AIP mangu', 'Maryhelp',
                'Immaculate hrt of mary', 'githunguri']

# read excel file again to get the sheet names
xl = pd.ExcelFile(path)

# store list of dataframes
dfs = []
# Loop through all facilities names and make a df
for facility in myFacilities:
    if facility in xl.sheet_names:
        df = pd.read_excel(path, sheet_name=facility)
        dfs.append(create_leap_data(df, facility))
    else:
        continue

## concatenate all dfs
df = pd.concat(dfs)

# drop weekly target rows
df = df.drop('Weekly target')
df = df[~df.index.str.contains('week')]

# insert pcr columns
df.insert(11, '1st PCR  (0 -<2 months)', 0)
df.insert(12, '1st PCR  (2 -<12 months)', 0)

# Reset index
df = df.reset_index()

# assign df 1 with date
df['Date'] = df[1]

col_name = "Date"
first_col = df.pop(col_name)

# insert date at position 0
df.insert(0, col_name, first_col)
# drop column named 1
df = df.drop(1, axis=1)

# Get the last report date
last_report_date = df['Date'].unique()[-1]
# filter df with the last day's report
df = df.loc[df['Date'] == last_report_date]

# sort df by facility name
df.sort_values('Facility', inplace=True)

lower_kiambu=['Thika L5', 'Ruiru', 'Gatundu L5H', 'kiandutu','langata','igegania', 'Ngoliba', 'jkuat', 'Makongeni', 
              'Kalimoni', 'Gachororo', 'AIP mangu', 'Maryhelp','Immaculate hrt of mary', 'githunguri']
upper_kiambu=['KCRH', 'LIMURU', 'WANGIGE', 'GITHUNGURI', 'KARURI', 'KIHARA', 'LARI', 'KIGUMO', 'GICHURU', 'LUSIGETTI',
                'NGEWA','UTHIRU', 'GITHIGA']

# Check if there is data missing
missing_facility=[]

# check whether to use lower or upper kiambu list

# option 1
# if "Thika L5" in df['Facility'].unique():

# option 2
if [facility for facility in lower_kiambu if facility in df['Facility'].unique()]:
    for facility in lower_kiambu:
        if facility in df['Facility'].unique():
            pass
        else:
            missing_facility.append(facility)
else:
    for facility in upper_kiambu:
        if facility in df['Facility'].unique():
            pass
        else:
            missing_facility.append(facility)

if len(missing_facility)!=0:
    print("============================================================================================================")
    if len(missing_facility)==1:
        print(f"{len(missing_facility)} FACILITY MISSING DATA")
    else:
        print(f"{len(missing_facility)} FACILITIES MISSING DATA")
    print(missing_facility)
    print("============================================================================================================")
else:
    print("============================================================================================================")
    print("NO DATA IS MISSING... LEAP REPORT IN EXCEL HAS BEEN GENERATED AND SAVED.")
    print("============================================================================================================")
    # Save data as Leap report excel
    df.to_excel("leap report.xlsx",index=False)
    
if "Thika L5" in df['Facility'].unique():
    print(f"\nLower kiambu should have {len(lower_kiambu)} LEAP sites. "
      f"\nYou generated data for {len(df['Facility'])} facilities.")
else:
     print(f"\nUpper kiambu should have {len(upper_kiambu)} LEAP sites."
           f"\nYou generated data for {len(df['Facility'])} facilities.")
df

NO DATA IS MISSING... LEAP REPORT IN EXCEL HAS BEEN GENERATED AND SAVED.

Upper kiambu should have 13 LEAP sites.
You generated data for 13 facilities.
Wall time: 5.24 s


indicator,Date,Facility,Tested (HTS_TST),Tested positive (HTS_POS),Tested under index testing (HTS_INDEX),Tested positive under index testing (HTS_POS_INDEX),Self testing kits issued (HTS_SELF),1st ANC Clients seen (PMTCT_STAT),1st ANC Clients seen (PMTCT_STAT),1st ANC Clients- KP,1st ANC Clients new positive,1st ANC on ART(PMTCT ART),1st PCR (0 -<2 months),1st PCR (2 -<12 months),TB cases new (TB STAT),TB cases new positive,TB cases KP,TB cases on ART (TB ART),PREP_New,PREP_CURR,GBV cases,Initiated on ART (TX_NEW),Current on ART (TX_CURR),CXCA,Returning to care (TX_RTC),Transfer In (TI),LTFU(>30 days),Transfer out(TO),Deceased
62,16th-22nd Aug,GICHURU,21,0,0,0,0,17,17,0,0,0,0,0,0,0,0,0,0,5,0,0,360,0,0,0,0,0,0
90,16th-22nd Aug,GITHIGA,10,0,1,1,6,11,11,0,0,0,0,0,1,0,0,2,0,9,0,1,251,0,1,2,1,1,0
27,16th-22nd Aug,GITHUNGURI,56,1,2,1,5,20,20,1,0,1,0,0,0,0,0,0,0,20,0,1,797,42,1,0,0,0,0
34,16th-22nd Aug,KARURI,39,4,6,0,0,20,20,1,0,0,0,0,1,0,0,0,2,21,1,0,727,7,0,0,1,0,0
6,16th-22nd Aug,KCRH,355,6,2,1,0,55,55,1,3,4,0,0,9,2,2,4,4,75,4,10,3032,7,3,0,2,1,0
55,16th-22nd Aug,KIGUMO,27,0,0,0,1,5,5,0,0,0,0,0,0,0,0,0,1,5,0,0,447,3,3,0,5,0,0
41,16th-22nd Aug,KIHARA,121,1,3,1,3,28,28,0,0,0,0,0,1,0,0,0,0,22,0,2,717,2,5,0,3,9,0
48,16th-22nd Aug,LARI,15,0,2,0,0,27,27,1,0,1,0,0,0,0,0,0,2,10,0,0,541,0,0,0,0,0,0
13,16th-22nd Aug,LIMURU,39,1,8,1,0,27,27,0,0,0,0,0,1,0,0,0,1,59,2,2,440,0,3,0,0,0,0
69,16th-22nd Aug,LUSIGETTI,15,1,1,0,0,14,14,0,0,0,0,0,0,0,0,0,0,0,0,0,321,0,0,0,0,0,0


## Troubleshoot specific facility data

In [1]:
import pandas as pd
# path='LEAP TEMPLATE FY21 WK 4 UPPER KIAMBU.xlsx' 
path='LEAP TEMPLATE FY21 Q3 WK 10 UPPER KIAMBU Updated.xlsx'

df = pd.read_excel(path, sheet_name='GICHURU')

# read fron the 3rd 
df = df.iloc[1:]
# Convert 1st row to column header
df.columns = df.iloc[0]
# Reset index
df.reset_index(drop=True, inplace=True)

# read fron the 3rd 
df = df.iloc[1:] 

# fill all nan with zero
df=df.fillna(0)




# drop all columns with zeros
df = df.loc[:, df.any()]

# drop all columns with "WEEK"
# df = df[(df.columns.drop(list(df.filter(regex='WEEK'or 'week'))))]
df = df[df.columns.drop(list(df.filter(regex='WEEK')))]
df = df[df.columns.drop(list(df.filter(regex='week')))]
df.columns

# rename the first column name
df.rename(columns={ df.columns[0]: "indicator" },inplace=True)

# create same indicators

mylist=[ 'Tested (HTS_TST)','Tested positive (HTS_POS)','Tested under index testing (HTS_INDEX)',
'Tested positive under index testing (HTS_POS_INDEX)','Self testing kits issued (HTS_SELF)',
'1st ANC Clients  seen (PMTCT_STAT)','1st ANC Clients- KP','1st ANC Clients new positive',
'1st  ANC  POS + KP (PMTCT POS)','1st ANC  on ART(PMTCT ART)','TB cases new (TB STAT)','TB cases new positive',
'TB cases  KP','TB cases  positive+KP (TB POS)','TB cases on ART (TB ART)',"0",'Initiated on ART (TX_NEW)',
'New on ART initiated  on TPT','Returning to care (TX_RTC)','Transfer In (TI)','Current on ART (TX_CURR)',
'TXML-treatment mortality and loss(ltfu+TO+Dead)','LTFU(>30 days)','Transfer out(TO)','Deceased','PREP_New','PREP_CURR',
'GBV cases ','CXCA']

mylist1=[ 'Tested (HTS_TST)','Tested positive (HTS_POS)','Tested under index testing (HTS_INDEX)',
    'Tested positive under index testing (HTS_POS_INDEX)','Self testing kits issued (HTS_SELF)',
    '1st ANC Clients  seen (PMTCT_STAT)','1st ANC Clients- KP','1st ANC Clients new positive',
    '1st  ANC  POS + KP (PMTCT POS)','1st ANC  on ART(PMTCT ART)','TB cases new (TB STAT)','TB cases new positive',
    'TB cases  KP','TB cases  positive+KP (TB POS)','TB cases on ART (TB ART)','Initiated on ART (TX_NEW)',
    'New on ART initiated  on TPT','Returning to care (TX_RTC)','Transfer In (TI)','Current on ART (TX_CURR)',
    'TXML-treatment mortality and loss(ltfu+TO+Dead)','LTFU(>30 days)','Transfer out(TO)','Deceased','PREP_New','PREP_CURR',
    'GBV cases ','CXCA']

mylist2=[ 'Tested (HTS_TST)', 'Tested positive (HTS_POS)','Tested under index testing (HTS_INDEX)',
       'Tested positive under index testing (HTS_POS_INDEX)','Self testing kits issued (HTS_SELF)',
        '1st ANC Clients  seen (PMTCT_STAT)','1st ANC Clients- KP', '1st ANC Clients new positive',
        '1st  ANC  POS + KP (PMTCT POS)','1st ANC  on ART(PMTCT ART)','TB cases new (TB STAT)',
        'TB cases new positive','TB cases  KP','TB cases  positive+KP (TB POS)','TB cases on ART (TB ART)',
        'Initiated on ART (TX_NEW)','New on ART initiated  on TPT','Returning to care (TX_RTC)',
        'Transfer In (TI)','Current on ART (TX_CURR)','TXML-treatment mortality and loss(ltfu+TO+Dead)',
        'LTFU(>30 days)','Transfer out(TO)','Deceased','PREP_New','PREP_CURR','GBV cases ',
        'CXCA','0',' ']


# create a new column
try:
    df['new_col'] = mylist
except:
    df['new_col'] = mylist1
    
# # replicate new column
# df['indicator']=df['new_col']
# # drop new column
# df=df.drop('new_col',axis=1)
# # fill all columns with nan with "type"
# df.columns = df.columns.fillna('type')
# # drop all columns with "type"
# df = df[df.columns.drop(list(df.filter(regex='type')))]
# # # # df.head()

# # set index to indicator
# df.set_index('indicator',inplace=True)

# # transpose df
# df=df.T 

# # Select column names to view
# col_needed=['Tested (HTS_TST)','Tested positive (HTS_POS)','Tested under index testing (HTS_INDEX)',
#             'Tested positive under index testing (HTS_POS_INDEX)','Self testing kits issued (HTS_SELF)',
#             '1st ANC Clients  seen (PMTCT_STAT)',
#             '1st ANC Clients- KP','1st ANC Clients new positive','1st ANC  on ART(PMTCT ART)',
#             'TB cases new (TB STAT)','TB cases new positive',
#             'TB cases  KP','TB cases on ART (TB ART)','PREP_New','PREP_CURR','GBV cases ',
#             'Initiated on ART (TX_NEW)','Current on ART (TX_CURR)','CXCA',
#             'Returning to care (TX_RTC)','Transfer In (TI)','LTFU(>30 days)','Transfer out(TO)','Deceased']
# df=df[col_needed]
# # drop all rows with zeros
# df=df.loc[~(df==0).all(axis=1)]
# # df.insert(0,'Facility',facility)

# df


In [ ]:
lower_kiambu = ['Thika L5', 'Ruiru', 'Gatundu L5H', 'kiandutu','langata','igegania', 'Ngoliba', 'jkuat', 'Makongeni', 
              'Kalimoni', 'Gachororo', 'AIP mangu', 'Maryhelp','Immaculate hrt of mary', 'githunguri']
L2 = ['WANGIGE','kiandutu']
if [facility for facility in lower_kiambu if i in df['Facility'].unique()]:
    print('true')